# Speech-to-Text with Whisper Transfer Learning

**Objective:** Fine-tune a Whisper base model on the United-Syn-Med dataset to improve medical speech transcription accuracy in a live teleconsultation context.

In [1]:
# Installing required packages

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer datasets torchaudio transformers accelerate soundfile

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ufl4m25o
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ufl4m25o
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━

In [2]:
# import dependent libraries

import os
import torch
import whisper
import pandas as pd
import soundfile as sf
from datasets import Dataset, DatasetDict
from jiwer import wer, cer
from transformers import WhisperProcessor, WhisperForConditionalGeneration, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torchaudio
from glob import glob
from tqdm import tqdm  # for progress bar

2025-06-15 00:27:48.928647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749947269.110560      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749947269.163075      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Loading the data
n = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if n < 3:
            print(os.path.join(dirname, filename))
            n += 1
        else: break
    if n >= 3: break

/kaggle/input/unitedsnymedsmall/unitedsynmed_small/transcript/validation.csv
/kaggle/input/unitedsnymedsmall/unitedsynmed_small/transcript/train.csv
/kaggle/input/unitedsnymedsmall/unitedsynmed_small/transcript/test.csv


In [4]:
# Paths to the dataset
audio_root = "/kaggle/input/unitedsnymedsmall/unitedsynmed_small/audio"
transcript_root = "/kaggle/input/unitedsnymedsmall/unitedsynmed_small/transcript/"

# Load CSVs and match them with audio paths
def load_split(split):
    csv_path = os.path.join(transcript_root, f"{split}.csv")
    df = pd.read_csv(csv_path)
    df["path"] = df["file_name"].apply(lambda x: os.path.join(audio_root, split, x))
    return df

# Create datasets
train_df = load_split("train")
test_df = load_split("test")
val_df = load_split("validation")

# Convert to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df),
    "validation": Dataset.from_pandas(val_df)
})


In [5]:
dataset["train"][:5]

{'file_name': ['drug-male-0b01f9d4-980d-451f-a8f1-18e899158859.wav',
  'drug-male-d58aac86-05d3-40ea-a61d-e1cbb7f3e790.wav',
  'drug-female-06c23421-e597-4cf4-a912-1d44c187a4f3.wav',
  'drug-male-9300288f-77c3-4c42-a0f6-166877f7f965.wav',
  'drug-female-86945722-12e1-4983-bf51-6aa27b196dc9.wav'],
 'transcription': ['Iron calx is a commonly used medicine to treat iron deficiency anemia.',
  'If you experience nausea or vomiting, DOMPAR may help alleviate your symptoms.',
  'AGROBEN-I is a reliable medicine for treating infections in plants.',
  "Make sure to follow your healthcare provider's instructions carefully while taking FEVIBID for optimal results.",
  'Clinical trials have shown favorable results with maralixibat chloride in pediatric patients.'],
 'path': ['/kaggle/input/unitedsnymedsmall/unitedsynmed_small/audio/train/drug-male-0b01f9d4-980d-451f-a8f1-18e899158859.wav',
  '/kaggle/input/unitedsnymedsmall/unitedsynmed_small/audio/train/drug-male-d58aac86-05d3-40ea-a61d-e1cbb7f3

In [6]:
# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
print(f"Using device: {device}")

Using device: cuda


In [ ]:

# # Load Whisper processor
# processor = WhisperProcessor.from_pretrained("openai/whisper-base")

# # Set target sample rate
# target_sample_rate = 16000

# def preprocess(batch):
#     audio_input, sr = sf.read(batch["path"])
    
#     # If the sample rate is not 16kHz, resample it
#     # if sr != target_sample_rate:
#     waveform = torch.tensor(audio_input, dtype=torch.float32).float().to(device) 
#     if len(waveform.shape) > 1 and waveform.shape[0] > 1:
#         waveform = waveform.mean(dim=0)  # Convert to mono
#     resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sample_rate).to(device)
#     audio_input = resampler(waveform).cpu().numpy()
    
#     inputs = processor(audio_input, sampling_rate=target_sample_rate, return_tensors="pt").to(device)
#     batch["input_features"] = inputs.input_features[0].to
#     batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
#     return batch

# # Apply preprocessing
# dataset = dataset.map(preprocess)

In [8]:
import torchaudio
from transformers import WhisperProcessor
import torch

# Initialize processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
device = "cuda" if torch.cuda.is_available() else "cpu"

def preprocess(batch):
    
    # 1. Load audio file
    waveform, sr = torchaudio.load(batch["path"])
    waveform = waveform.to(device)  # Move to GPU here
    
    # 2. Verify sample rate (optional if you're certain)
    if sr != 16000:
        raise ValueError(f"Invalid sample rate {sr}Hz (expected 16000Hz)")
    
    # 3. Convert to mono if needed
    if waveform.dim() > 1 and waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    
    # 4. Process audio - key fix is making sure we use the tensor, not the method
    audio_array = waveform.squeeze().cpu().numpy()  # Explicitly move to CPU first
    
    # 5. Generate features
    inputs = processor(
        audio_array,
        sampling_rate=16000,
        return_tensors="pt"
    )
    
    # 6. Prepare output - ensure we're using the actual tensors
    batch["input_features"] = inputs.input_features[0].numpy()  # Convert to numpy array
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    batch["input_features"] = inputs.input_features[0].cpu().numpy()
    
    return batch
        

# Apply preprocessing
dataset = dataset.map(preprocess)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [9]:
#save the preprocessed dataset

# Define your output directory in Kaggle's working directory
output_dir = "/kaggle/working/preprocessed_dataset"

# Create directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the dataset
dataset.save_to_disk(output_dir)

print(f"✅ Dataset saved to {output_dir}")

Saving the dataset (0/19 shards):   0%|          | 0/9500 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

✅ Dataset saved to /kaggle/working/preprocessed_dataset


In [ ]:
# #Reload dataset incase kernel dies

# from datasets import load_from_disk

# # Load the saved dataset
# dataset = load_from_disk("/kaggle/working/preprocessed_dataset")

# # Verify it loaded correctly
# print(dataset)
# print(dataset["train"][0])  # Check a sample

In [10]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100)
        batch["labels"] = labels

        return batch

In [11]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
# Freeze encoder layers
for param in model.model.encoder.parameters():
    param.requires_grad = False

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [13]:
from transformers.integrations import TensorBoardCallback

# Create logging directory if it doesn't exist
os.makedirs("./logs", exist_ok=True)

training_args = TrainingArguments(
    output_dir="./whisper-medical",
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_strategy="steps",       # New: Log at intervals
    logging_steps=50,              # Log every 50 steps
    learning_rate=1e-4,
    warmup_steps=500,
    fp16=True,
    push_to_hub=False,
    report_to="tensorboard",       # New: Enable TensorBoard
    load_best_model_at_end=True,   # New: Useful for tracking
    metric_for_best_model="eval_loss",
)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
)


/tmp/ipykernel_35/2233242723.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:

# training_args = TrainingArguments(
#     output_dir="./whisper-medical",
#     per_device_train_batch_size=8,
#     eval_strategy="epoch",  # Changed from evaluation_strategy
#     save_strategy="epoch",
#     num_train_epochs=5,
#     logging_dir="./logs",
#     learning_rate=1e-4,
#     warmup_steps=500,
#     fp16=True,
#     push_to_hub=False,
# )
# data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     tokenizer=processor.feature_extractor,
#     data_collator=data_collator,
# )

/tmp/ipykernel_35/2511081694.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# Confirm GPU is being used

print(f"GPU available: {torch.cuda.is_available()}")
print(f"Device being used: {trainer.args.device}")  # After creating trainer

GPU available: True
Device being used: cuda:0


In [15]:
# Check GPU memory usage

!nvidia-smi  # Works in Kaggle notebooks

Sun Jun 15 00:35:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             32W /   70W |     399MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
# Train Model

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.385600,0.401000
2,0.216000,0.358396
3,0.087100,0.352310
4,0.030000,0.356789
5,0.008800,0.356194


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gath

TrainOutput(global_step=2970, training_loss=0.19543793970085555, metrics={'train_runtime': 7939.7311, 'train_samples_per_second': 5.983, 'train_steps_per_second': 0.374, 'total_flos': 3.0808498176e+18, 'train_loss': 0.19543793970085555, 'epoch': 5.0})

In [1]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer_score = wer(label_str, pred_str)
    cer_score = cer(label_str, pred_str)

    return {"wer": wer_score, "cer": cer_score}

results = trainer.evaluate()
print(results)

NameError: name 'trainer' is not defined